# General Example
In this example we show how to load a benchmark dataset, how to train a model on it and which are the different types of evaluation protocols that we can use.

In [6]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

## Load the dataset

In [7]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

## Train the model

In [8]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# you can continue training from where you left after restoring the model
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./transe_train_logs')

# create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=5,
                                   k=300,
                                   scoring_type='TransE')

# you can either use optimizers/regularizers/loss/initializers with default values or you can 
# import it and customize the hyperparameters and pass it to compile

# Let's create an adam optimizer with customized learning rate =0.005
adam = tf.keras.optimizers.Adam(learning_rate=0.005)
# Let's compile the model with self_advarsarial loss of default parameters
model.compile(optimizer=adam, loss='self_adversarial')

# fit the model to data.
model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             callbacks=[tensorboard_callback])

# the training can be visualised using the following command:
# tensorboard --logdir='./transe_train_logs' --port=8891 
# open the browser and go to the following URL: http://127.0.0.1:8891/

Epoch 1/10
29/29 [==============================] - 7s 233ms/step - loss: 36773.3906
Epoch 2/10
29/29 [==============================] - 6s 215ms/step - loss: 22626.3574
Epoch 3/10
29/29 [==============================] - 6s 202ms/step - loss: 17343.5254
Epoch 4/10
29/29 [==============================] - 6s 198ms/step - loss: 14640.1602
Epoch 5/10
29/29 [==============================] - 6s 223ms/step - loss: 13013.9121
Epoch 6/10
29/29 [==============================] - 6s 202ms/step - loss: 11937.1406
Epoch 7/10
29/29 [==============================] - 6s 194ms/step - loss: 11176.6133
Epoch 8/10
29/29 [==============================] - 6s 213ms/step - loss: 10612.2920
Epoch 9/10
29/29 [==============================] - 6s 196ms/step - loss: 10177.2930
Epoch 10/10
29/29 [==============================] - 6s 200ms/step - loss: 9833.4297


## Predict scores

In [9]:
pred = model.predict(dataset['test'][:5], 
                       batch_size=100)
pred

array([-2.266111 , -2.095364 , -2.289988 , -3.8879156, -4.619501 ],
      dtype=float32)

## Evaluate the model (without filter)

### Subject and object (s,o) corruption
This evaluation protocol consists in creating corrupted triples via the corruption of both subject and object of existing triples. This is considered the standard protocol for evaluation in Knowledge Graph Embedding models literature.

In [10]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'],     # test set
                       batch_size=100,      # evaluation batch size
                       corrupt_side='s,o'   # sides to corrupt for scoring and ranking
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 46s 223ms/step
MR: 472.3101575496624
MRR: 0.08900155667769008
hits@1: 0.0
hits@10: 0.2428319796457579


### Object corruption
Corruptions are generated by changing only the object of triples.

In [11]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], 
                       batch_size=100, 
                       corrupt_side='o' # corrupt only the object
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 25s 123ms/step
MR: 268.11287797240436
MRR: 0.12974471587250763
hits@1: 0.0
hits@10: 0.3512085331245719


### Subject corruption
Corruptions are generated by changing only the subject of triples.

In [12]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], 
                       batch_size=100, 
                       corrupt_side='s' # corrupt only the subject
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 27s 129ms/step
MR: 676.5074371269204
MRR: 0.048258397482872535
hits@1: 0.0
hits@10: 0.13445542616694392


## Evaluation with Filters
Triples specified inside the filter are removed from the corruptions that are generated to avoid the creation of false negatives.

In [13]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], 
                       batch_size=100, 
                       corrupt_side='s,o', # corrupt both subject and object
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 929s 5s/step
MR: 364.22042274195127
MRR: 0.19061439893688528
hits@1: 0.12329973578628045
hits@10: 0.32199823857520304


## Evaluation using a subset of entities for corruption
Specify a subset of entities that are used for corrupting, depending on the evaluation strategy chosen, either the subject, the object or both. Notice that, despite not being the standard evaluation protocol, using a subset of entities can generate more meaningful corruptions and also reduce a lot the computational overhead caused by sampling corruptions among all the entities in the knowledge graph.

In [14]:
# Let's get all the month present in training
months = set(dataset['train'][
    dataset['train'][:, 1] == 
        '/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month'][:, 2])
len(months)

12

In [15]:
# consider we are evaluating the below test set which is specific to one predicate
# This predicate tells the best time of the year(o) to visit a destination (s)
dest_month_test_triples = dataset['test'][
    dataset['test'][:, 1] ==
        '/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month']

In [16]:
# Let's say we want to evaluate this test set by corrupting the object with only months.
# we can pass the months as entities_subset and generate corruptions only using this subset 
# instead of all entities in the graph
# This approach is very useful when the graph size is big and/or 
# when our hypothesis belongs to a specific predicate type
# When graph size is big we can randomly sample fixed number of small subset of entities and use it as corruption

# evaluate on the test set
ranks = model.evaluate(dest_month_test_triples, 
                       batch_size=100, 
                       corrupt_side='o', # corrupt only the object
                       entities_subset=months,
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

2/2 [==============================] - 2s 1s/step
MR: 1.0833333333333333
MRR: 0.9861111111111112
hits@1: 0.9833333333333333
hits@10: 1.0


## Visualize the embeddings

In [17]:
from ampligraph.utils import create_tensorboard_visualizations

In [18]:
create_tensorboard_visualizations(model, 
                                  entities_subset=['/m/027rn', '/m/06cx9', '/m/017dcd', '/m/06v8s0', '/m/07s9rl0'], 
                                  labels=['ent1', 'ent2', 'ent3', 'ent4', 'ent5'],
                                  loc = './selected_subset_embeddings_vis')


In [ ]:
create_tensorboard_visualizations(model, 
                                  entities_subset='all',
                                  loc = './full_embeddings_vis')

# the embeddings can be visualised uncommenting the following command:
# ! tensorboard --logdir='./full_embeddings_vis' --port=8891 
# open the browser and go to the following URL: http://127.0.0.1:8891/#projector